In [1]:
#installing dependencies

!pip install tensorflow==2.9.1 tensorflow-gpu==2.9.1 opencv-python matplotlib

In [2]:
#importing dependencies

import cv2
import os
import random
import numpy as np
from matplotlib import pyplot as plt

In [3]:
# importing tensorflow dependencies - Functional API
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Conv2D, Dense, MaxPooling2D,Input, Flatten
import tensorflow as tf

In [4]:
# To avoid out of memory errors by setting GPU Memory Consumption Growth

#accessing all the gpus on the machine
gpus = tf.config.experimental.list_physical_devices("GPU") 
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu,True)


In [5]:
# setting up paths
POS_PATH = os.path.join('data', 'positive')
NEG_PATH = os.path.join('data', 'negative')
ANC_PATH = os.path.join('data', 'anchor')


In [129]:
# making the directories

os.makedirs(POS_PATH)
os.makedirs(NEG_PATH)
os.makedirs(ANC_PATH)

In [ ]:
# Wild dataset (http://vis-www.cs.umass.edu/lfw/)
# uncompressing the Tar GZ Labelled Faces in the Wild Dataset

!tar -xf lfw.tgz

In [ ]:
# moving LFW images to data/negative repository

for directory in os.listdir('lfw'):
    for file in os.listdir(os.path.join('lfw',directory)):
        EX_PATH = os.path.join('lfw',directory,file)
        NEW_PATH = os.path.join(NEG_PATH, file)
        os.replace(EX_PATH, NEW_PATH)


In [6]:
# importing uuid library to generate unique image names
import uuid

In [7]:
# colleting positive and anchor classes

#establishing a connection to the webcam
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    
    # cutting down frame to 250x250
    frame = frame[120:120+250,200:200+250, :]
    
    #collecting anchors
    if cv2.waitKey(1) & 0XFF == ord('a'):
        # Creating unique file path 
        imgname = os.path.join(ANC_PATH, '{}.jpg'.format(uuid.uuid1()))
        cv2.imwrite(imgname, frame)
        
    #collecting positives
    if cv2.waitKey(1) & 0XFF == ord('p'):
        # Create the unique file path 
        imgname = os.path.join(POS_PATH, '{}.jpg'.format(uuid.uuid1()))
        cv2.imwrite(imgname, frame)
    
    #showing image back to the screen
    cv2.imshow("Image Collection", frame)
    #breaking
    if cv2.waitKey(1) & 0XFF == ord('q'):
        break
        
# releasing the webcam
cap.release()
# for closing the image show frame
cv2.destroyAllWindows()

## Data Augmentation

In [ ]:
def data_aug(img):
    data = []
    for i in range(9):
        img = tf.image.stateless_random_brightness(img, max_delta=0.02, seed=(1,2))
        img = tf.image.stateless_random_contrast(img, lower=0.6, upper=1, seed=(1,3))
        # img = tf.image.stateless_random_crop(img, size=(20,20,3), seed=(1,2))
        img = tf.image.stateless_random_flip_left_right(img, seed=(np.random.randint(100),np.random.randint(100)))
        img = tf.image.stateless_random_jpeg_quality(img, min_jpeg_quality=90, max_jpeg_quality=100, seed=(np.random.randint(100),np.random.randint(100)))
        img = tf.image.stateless_random_saturation(img, lower=0.9,upper=1, seed=(np.random.randint(100),np.random.randint(100)))
            
        data.append(img)
    
    return data

In [1]:
import os
import uuid

In [ ]:
img_path = os.path.join(ANC_PATH, '924e839c-135f-11ec-b54e-a0cec8d2d278.jpg')
img = cv2.imread(img_path)
augmented_images = data_aug(img)

for image in augmented_images:
    cv2.imwrite(os.path.join(ANC_PATH, '{}.jpg'.format(uuid.uuid1())), image.numpy())

In [ ]:
for file_name in os.listdir(os.path.join(POS_PATH)):
    img_path = os.path.join(POS_PATH, file_name)
    img = cv2.imread(img_path)
    augmented_images = data_aug(img) 
    
    for image in augmented_images:
        cv2.imwrite(os.path.join(POS_PATH, '{}.jpg'.format(uuid.uuid1())), image.numpy())

In [8]:
# anchor, negative, positive created!!

In [9]:
#getting image directories
anchor = tf.data.Dataset.list_files(ANC_PATH+'\*.jpg').take(3000)
positive = tf.data.Dataset.list_files(POS_PATH+'\*.jpg').take(3000)
negative = tf.data.Dataset.list_files(NEG_PATH+'\*.jpg').take(3000)

In [10]:
dir_test = anchor.as_numpy_iterator()

In [11]:
print(dir_test.next())

b'data\\anchor\\6db9cdd0-fb13-11ec-a668-002b67e194ec.jpg'


In [63]:
# preprocessing -- scale and resize

def preprocess(file_path):
    # reading image from file path
    byte_img = tf.io.read_file(file_path)
    #loading the image
    img = tf.io.decode_jpeg(byte_img)
    # preprocessing steps - resizing the image to be 100x100x3
    img = tf.image.resize(img,(100,100))
    # scaling img b/w 0 and 1
    img = img/255.0
    return img

In [64]:
# Creating a Labelled Datatset

# (anchor, positive) => 1,1,1,1,1
# (anchor, negative) => 0,0,0,0,0

In [65]:
positives = tf.data.Dataset.zip((anchor,positive,tf.data.Dataset.from_tensor_slices(tf.ones(len(anchor)))))
negatives = tf.data.Dataset.zip((anchor,negative,tf.data.Dataset.from_tensor_slices(tf.ones(len(anchor)))))
data = positives.concatenate(negatives)

In [66]:
samples = data.as_numpy_iterator()

In [67]:
examples = samples.next()

In [68]:
examples

(b'data\\anchor\\6ca4c11c-fb13-11ec-9370-002b67e194ec.jpg',
 b'data\\positive\\8cc859a4-fb13-11ec-83e5-002b67e194ec.jpg',
 1.0)

In [69]:
# Creating Train and Test Partition

def preprocess_twin(input_img, validation_img, label):
    return(preprocess(input_img),preprocess(validation_img),label)

In [78]:
res = preprocess_twin(*examples)
res

(<tf.Tensor: shape=(100, 100, 3), dtype=float32, numpy=
 array([[[0.8235294 , 0.79607844, 0.7647059 ],
         [0.8245098 , 0.7970588 , 0.7656863 ],
         [0.8333333 , 0.8       , 0.77156866],
         ...,
         [0.90563726, 0.8860294 , 0.8625    ],
         [0.90416664, 0.8845588 , 0.8610294 ],
         [0.904902  , 0.88529414, 0.8617647 ]],
 
        [[0.82843137, 0.79901963, 0.7705882 ],
         [0.83431375, 0.80490196, 0.7764706 ],
         [0.8394608 , 0.80563724, 0.7794118 ],
         ...,
         [0.90906864, 0.8894608 , 0.8659314 ],
         [0.90612745, 0.8865196 , 0.8629902 ],
         [0.9019608 , 0.88235295, 0.85882354]],
 
        [[0.8352941 , 0.8       , 0.7745098 ],
         [0.8401961 , 0.80490196, 0.7794118 ],
         [0.84607846, 0.81078434, 0.7852941 ],
         ...,
         [0.91495097, 0.8953431 , 0.8718137 ],
         [0.9127451 , 0.8931373 , 0.86960787],
         [0.9127451 , 0.8931373 , 0.86960787]],
 
        ...,
 
        [[0.9078431 , 0.9       

In [71]:
# Building dataloader pipeline

data = data.map(preprocess_twin)
data = data.cache()
data = data.shuffle(buffer_size = 10000)

In [79]:
# Training Partition

train_data = data.take(round(len(data)*0.7))
train_data = train_data.batch(16)
train_data = train_data.prefetch(8)

In [81]:
train_samples = train_data.as_numpy_iterator()
train_sample = train_samples.next()
len(train_sample[0])

16

In [87]:
# Testing Partition

test_data = data.skip(round(len(data)*0.7))
test_data = test_data.take(round(len(data)*0.3))
test_data = test_data.batch(16)
test_data = test_data.prefetch(8)

## MODEL ENGINEERING

In [92]:
inp = Input(shape=(100,100,3), name='input_image')
    
#First Block
c1 = Conv2D(64,(10,10), activation='relu')(inp)
m1 = MaxPooling2D(64, (2,2), padding='same')(c1)
    
# Second block
c2 = Conv2D(128, (7,7), activation='relu')(m1)
m2 = MaxPooling2D(64, (2,2), padding='same')(c2)
    
# Third block 
c3 = Conv2D(128, (4,4), activation='relu')(m2)
m3 = MaxPooling2D(64, (2,2), padding='same')(c3)
    
# Final embedding block
c4 = Conv2D(256, (4,4), activation='relu')(m3)
f1 = Flatten()(c4)
d1 = Dense(4096, activation='sigmoid')(f1)
    
mod = Model(inputs=[inp],outputs=[d1] ,name='embedding')

In [93]:
mod.summary()

Model: "embedding"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_image (InputLayer)    [(None, 100, 100, 3)]     0         
                                                                 
 conv2d_4 (Conv2D)           (None, 91, 91, 64)        19264     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 46, 46, 64)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 40, 40, 128)       401536    
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 20, 20, 128)      0         
 2D)                                                             
                                                                 
 conv2d_6 (Conv2D)           (None, 17, 17, 128)       26

In [89]:
# Building embedding layer  (according to paper (pg.4))

def make_embedding():
    inp = Input(shape=(100,100,3), name='input_image')
    
    #First Block
    c1 = Conv2D(64,(10,10), activation='relu')(inp)
    m1 = MaxPooling2D(64, (2,2), padding='same')(c1)
    
    # Second block
    c2 = Conv2D(128, (7,7), activation='relu')(m1)
    m2 = MaxPooling2D(64, (2,2), padding='same')(c2)
    
    # Third block 
    c3 = Conv2D(128, (4,4), activation='relu')(m2)
    m3 = MaxPooling2D(64, (2,2), padding='same')(c3)
    
    # Final embedding block
    c4 = Conv2D(256, (4,4), activation='relu')(m3)
    f1 = Flatten()(c4)
    d1 = Dense(4096, activation='sigmoid')(f1)
    
    return Model(inputs=[inp],outputs=[d1] ,name='embedding')

In [94]:
embedding = make_embedding()

In [95]:
embedding.summary()

Model: "embedding"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_image (InputLayer)    [(None, 100, 100, 3)]     0         
                                                                 
 conv2d_8 (Conv2D)           (None, 91, 91, 64)        19264     
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 46, 46, 64)       0         
 2D)                                                             
                                                                 
 conv2d_9 (Conv2D)           (None, 40, 40, 128)       401536    
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 20, 20, 128)      0         
 2D)                                                             
                                                                 
 conv2d_10 (Conv2D)          (None, 17, 17, 128)       26

In [97]:
# Building Distance Layer


#creating L1 Distance class
class L1Dist(Layer):
    # Inheritance
    def __init__(self,**kwargs):
        super().__init__()
    
    # similarity calculation
    def cell(self, input_embedding, validation_embedding):
        return tf.math.abs(input_embedding - validation_embedding)

In [100]:
# Making Siamese Model
# siamese neural network
def make_siamese_model(): 
    
    # Anchor image input in the network
    input_image = Input(name='input_img', shape=(100,100,3))
    
    # Validation image in the network 
    validation_image = Input(name='validation_img', shape=(100,100,3))
    
    # Combining siamese distance components
    siamese_layer = L1Dist()
    siamese_layer._name = 'distance'
    distances = siamese_layer(embedding(input_image), embedding(validation_image))
    
    # Classification layer 
    classifier = Dense(1, activation='sigmoid')(distances)
    
    return Model(inputs=[input_image, validation_image], outputs=classifier, name='SiameseNetwork')

In [101]:
siamese_model = make_siamese_model()

In [102]:
siamese_model.summary()

Model: "SiameseNetwork"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_img (InputLayer)         [(None, 100, 100, 3  0           []                               
                                )]                                                                
                                                                                                  
 validation_img (InputLayer)    [(None, 100, 100, 3  0           []                               
                                )]                                                                
                                                                                                  
 embedding (Functional)         (None, 4096)         38960448    ['input_img[0][0]',              
                                                                  'validation_img[0][

## TRAINING OUR MODEL

In [103]:
# SETTING UP LOSS AND OPTIMIZER
# tf.losses.BinaryCrossentropy??

binary_cross_loss = tf.losses.BinaryCrossentropy()

In [104]:
opt = tf.keras.optimizers.Adam(1e-4)

In [105]:
# ESTABLISHING CHECKPOINTS

checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt')
checkpoint = tf.train.Checkpoint(opt=opt, siamese_model=siamese_model)

In [118]:
# BUILDING TRAIN STEP FUNCTION

#The basic flow for training of one batch:
#    1.Make a prediction
#    2.Calculate loss
#   3.Derive gradients
#   4.Calculate new weights and apply

In [112]:
test_batch = train_data.as_numpy_iterator()

In [113]:
batch_1 = test_batch.next()

In [114]:
X = batch_1[:2]

In [120]:
np.array(X).shape

(2, 16, 100, 100, 3)

In [115]:
y = batch_1[2]

In [116]:
y

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
      dtype=float32)

In [119]:
@tf.function
def train_step(batch):
    
    # Recording all of our operations 
    with tf.GradientTape() as tape:     
        # Getting anchor and positive/negative image
        X = batch[:2]
        # Getting our label
        y = batch[2]
        
        # Forward pass
        yhat = siamese_model(X, training=True)
        # Calculating loss
        loss = binary_cross_loss(y, yhat)
    print(loss)
        
    # Calculating gradients
    grad = tape.gradient(loss, siamese_model.trainable_variables)
    
    # Calculating updated weights and apply to siamese model
    opt.apply_gradients(zip(grad, siamese_model.trainable_variables))
  
    return loss

In [123]:
# BUILDING THE TRAINING LOOP

def train(data, EPOCHS):
    # Looping through epochs
    for epoch in range(1, EPOCHS+1):
        print('\n Epoch {}/{}'.format(epoch, EPOCHS))
        progbar = tf.keras.utils.Progbar(len(data))
        
        # Looping through each batch
        for idx, batch in enumerate(data):
            # Running train step 
            train_step(batch)
            progbar.update(idx+1)
        
        # Saving checkpoints
        if epoch % 10 == 0: 
            checkpoint.save(file_prefix=checkpoint_prefix)

In [122]:
# TRAINING THE MODEL

EPOCHS = 50
train(train_data, EPOCHS)


 Epoch 1/50
Tensor("binary_crossentropy/weighted_loss/value:0", shape=(), dtype=float32)
Tensor("binary_crossentropy/weighted_loss/value:0", shape=(), dtype=float32)
21/21 [==============================] - 256s 12s/step

 Epoch 2/50
21/21 [==============================] - 260s 12s/step

 Epoch 3/50
21/21 [==============================] - 256s 12s/step

 Epoch 4/50
21/21 [==============================] - 253s 12s/step

 Epoch 5/50
21/21 [==============================] - 251s 12s/step

 Epoch 6/50
21/21 [==============================] - 257s 12s/step

 Epoch 7/50
21/21 [==============================] - 255s 12s/step

 Epoch 8/50
21/21 [==============================] - 256s 12s/step

 Epoch 9/50
21/21 [==============================] - 261s 12s/step

 Epoch 10/50
21/21 [==============================] - 256s 12s/step

 Epoch 11/50
21/21 [==============================] - 260s 12s/step

 Epoch 12/50
21/21 [==============================] - 259s 12s/step

 Epoch 13/50
21/21 [======

## EVALUATING OUR MODEL

In [124]:
#Importing metric calculations

from tensorflow.keras.metrics import Precision, Recall

In [125]:
# Getting a batch of data
test_input, test_val, y_true = test_data.as_numpy_iterator().next()

In [126]:
test_var = test_data.as_numpy_iterator().next()

In [128]:
test_var[2]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
      dtype=float32)

In [130]:
# Making predictions
y_hat = siamese_model.predict([test_input, test_val])
y_hat

1/1 [==============================] - 1s 921ms/step


array([[1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.]], dtype=float32)

In [131]:
# Post processing the results 
[1 if prediction > 0.5 else 0 for prediction in y_hat ]

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [132]:
y_true

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
      dtype=float32)

In [133]:
# CALCULATING METRICS

# Creating a metric object 
m = Recall()

# Calculating the recall value 
m.update_state(y_true, y_hat)

# Returning Recall Result
m.result().numpy()

1.0

In [134]:
# Creating a metric object 
m = Precision()

# Calculating the precision value 
m.update_state(y_true, y_hat)

# Returning precision Result
m.result().numpy()

1.0

In [ ]:
r = Recall()
p = Precision()

for test_input, test_val, y_true in test_data.as_numpy_iterator():
    yhat = siamese_model.predict([test_input, test_val])
    r.update_state(y_true, yhat)
    p.update_state(y_true, yhat)

print(r.result().numpy(),p.result().numpy())

In [144]:
    # RESULT VISUALIZATION

# Setting plot size 
    #plt.figure(figsize=(10,8))

# Setting first subplot
    #plt.subplot(1,2,1)
    #plt.imshow(test_input[0])

# Setting second subplot
    #plt.subplot(1,2,2)
    #plt.imshow(test_val[0])

    #plt.show()

## SAVING MODEL

In [136]:
# Saving weights
siamese_model.save('siamesemodel.h5')

In [137]:
L1Dist

__main__.L1Dist

In [138]:
# Reloading our model 
model = tf.keras.models.load_model('siamesemodel.h5', 
                                   custom_objects={'L1Dist':L1Dist, 
                                                   'BinaryCrossentropy':tf.losses.BinaryCrossentropy})

In [139]:
# Making predictions with reloaded model
model.predict([test_input, test_val])

1/1 [==============================] - 1s 552ms/step


array([[1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.]], dtype=float32)

In [140]:
# Model summary
model.summary()

Model: "SiameseNetwork"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_img (InputLayer)         [(None, 100, 100, 3  0           []                               
                                )]                                                                
                                                                                                  
 embedding (Functional)         (None, 4096)         38960448    ['input_img[0][0]']              
                                                                                                  
 l1_dist_2 (L1Dist)             (None, 4096)         0           ['embedding[0][0]']              
                                                                                                  
 validation_img (InputLayer)    [(None, 100, 100, 3  0           []                  

## REAL TIME TESTING


In [141]:
os.listdir(os.path.join('application_data', 'verification_images'))

['7ccf7617-fb13-11ec-81a2-002b67e194ec.jpg',
 '7dcb1f30-fb13-11ec-844b-002b67e194ec.jpg',
 '7e8fba3c-fb13-11ec-ad66-002b67e194ec.jpg',
 '7f3faeb7-fb13-11ec-a0f9-002b67e194ec.jpg',
 '82e861d4-fb13-11ec-b532-002b67e194ec.jpg',
 '83df40fb-fb13-11ec-ad64-002b67e194ec.jpg',
 '85269ad9-fb13-11ec-b199-002b67e194ec.jpg',
 '861d6295-fb13-11ec-89d1-002b67e194ec.jpg',
 '864a79d4-fb13-11ec-9017-002b67e194ec.jpg',
 '8722efb6-fb13-11ec-b80d-002b67e194ec.jpg',
 '87904331-fb13-11ec-88c6-002b67e194ec.jpg',
 '87c44817-fb13-11ec-b990-002b67e194ec.jpg',
 '87f1ae1e-fb13-11ec-9e21-002b67e194ec.jpg',
 '88f749cd-fb13-11ec-b8ae-002b67e194ec.jpg',
 '891f8720-fb13-11ec-9392-002b67e194ec.jpg',
 '89c60237-fb13-11ec-9eb5-002b67e194ec.jpg',
 '8abca5bd-fb13-11ec-92b3-002b67e194ec.jpg',
 '8ae4eb25-fb13-11ec-8046-002b67e194ec.jpg',
 '8b6ce4ef-fb13-11ec-8a90-002b67e194ec.jpg',
 '8bf9f016-fb13-11ec-9e2c-002b67e194ec.jpg',
 '92a657d6-fb13-11ec-9f4a-002b67e194ec.jpg',
 '92f69e15-fb13-11ec-88a3-002b67e194ec.jpg',
 '934ccb4d

In [142]:
for image in os.listdir(os.path.join('application_data', 'verification_images')):
    validation_img = os.path.join('application_data', 'verification_images', image)
    print(validation_img)

application_data\verification_images\7ccf7617-fb13-11ec-81a2-002b67e194ec.jpg
application_data\verification_images\7dcb1f30-fb13-11ec-844b-002b67e194ec.jpg
application_data\verification_images\7e8fba3c-fb13-11ec-ad66-002b67e194ec.jpg
application_data\verification_images\7f3faeb7-fb13-11ec-a0f9-002b67e194ec.jpg
application_data\verification_images\82e861d4-fb13-11ec-b532-002b67e194ec.jpg
application_data\verification_images\83df40fb-fb13-11ec-ad64-002b67e194ec.jpg
application_data\verification_images\85269ad9-fb13-11ec-b199-002b67e194ec.jpg
application_data\verification_images\861d6295-fb13-11ec-89d1-002b67e194ec.jpg
application_data\verification_images\864a79d4-fb13-11ec-9017-002b67e194ec.jpg
application_data\verification_images\8722efb6-fb13-11ec-b80d-002b67e194ec.jpg
application_data\verification_images\87904331-fb13-11ec-88c6-002b67e194ec.jpg
application_data\verification_images\87c44817-fb13-11ec-b990-002b67e194ec.jpg
application_data\verification_images\87f1ae1e-fb13-11ec-9e21-002

In [145]:
# Verification Function

def verify(model, detection_threshold, verification_threshold):
    # Building results array
    results = []
    for image in os.listdir(os.path.join('application_data', 'verification_images')):
        input_img = preprocess(os.path.join('application_data', 'input_image', 'input_image.jpg'))
        validation_img = preprocess(os.path.join('application_data', 'verification_images', image))
        
        # Making Predictions 
        result = model.predict(list(np.expand_dims([input_img, validation_img], axis=1)))
        results.append(result)
    
    # Detection Threshold: Metric above which a prediciton is considered positive 
    detection = np.sum(np.array(results) > detection_threshold)
    
    # Verification Threshold: Proportion of positive predictions / total positive samples 
    verification = detection / len(os.listdir(os.path.join('application_data', 'verification_images'))) 
    verified = verification > verification_threshold
    
    return results, verified

In [146]:
# OpenCV REAL TIME VERIFICATION

In [151]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    frame = frame[120:120+250,200:200+250, :]
    
    cv2.imshow('Verification', frame)
    
    # Verification trigger
    if cv2.waitKey(10) & 0xFF == ord('v'):
        # Save input image to application_data/input_image folder 
        cv2.imwrite(os.path.join('application_data', 'input_image', 'input_image.jpg'), frame)
        # Running for verification
        results, verified = verify(model, 0.9, 0.7)
        print(verified)
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 163ms/step
True
1/1 [==============================] - 0s 159ms/step
True


In [152]:
np.sum(np.squeeze(results) > 0.9)

50

In [153]:
50/50

1.0

In [ ]:
# THE END 